### Project Title: Analyzing and Predicting Commuting Choices of Graduate Students

**Objective**:

We are analyzing the commuting preferences of graduate students during the winter season when only driving or taking the bus are feasible options due to severe weather conditions. This study will assist the university in optimizing parking and bus transportation needs during the winter months.

Additionally, we are investigating the commuting choices of graduate students in the spring, when biking and walking become viable alternatives. This research will support the university in planning for car parking, bike rack availability, and bus transportation during the spring season. Moreover, the university is considering adjustments to bus routes and aims to understand how these changes may influence students' commuting decisions.

**Data:**

The analysis utilizes two datasets, both derived from simulated data on the commuting choices of 1,000 graduate students who commute to campus from distances greater than one mile. These datasets can be found [here](https://github.com/Oishika-Kar/Analyzing-and-Predicting-Commuting-Choices-of-Graduate-Students/blob/3652e98d42cb97087f1ec622f69a43ee31ff9546/commute_datasets.zip.zip). Detailed descriptions of the variables in each dataset are available in the "commute_descriptions.txt" file.

1. **commute_binary.csv**: This dataset contains information on students' commuting choices during the winter season, when only two options are feasible due to severe weather conditions: driving a car or taking a bus. This dataset will help the university plan for parking and bus transportation needs during the winter months.

2. **commute_multinomial.csv**: This dataset includes data on commuting choices during the spring season, when weather conditions allow for additional options. Students can choose from four commuting modes: driving a car, taking a bus, biking, or walking. This dataset will aid the university in planning for car parking, bike rack availability, and bus transportation during the spring. Additionally, the university is considering adjustments to bus routes and seeks to understand how these changes might influence students' commuting decisions.

In [1]:
#import necessary packages
import statsmodels.api as sm
import statsmodels as sms
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
! pip install xlogit
import xlogit
from xlogit.utils import wide_to_long
from xlogit import MixedLogit
from xlogit import MultinomialLogit

In [63]:
data = pd.read_csv("commute_binary.csv")
data.head(15)

,id,mode,time.car,cost.car,time.bus,cost.bus,price_gas,snowfall,construction,bus_detour,age,income,marital_status
0,1,car,13,0.79,26,0,2.44,0.0,0,1,33,28,married
1,2,car,9,0.59,14,0,2.59,0.0,0,0,25,28,single
2,3,bus,20,1.62,35,0,2.55,1.0,0,0,27,23,single
3,4,bus,11,0.30,11,0,2.53,2.2,0,0,25,21,single
4,5,bus,12,0.44,15,0,2.49,11.2,0,0,26,26,married
5,6,bus,9,0.23,10,0,2.46,6.5,0,0,28,24,married
6,7,bus,13,0.54,17,0,2.58,1.9,0,0,25,27,single
7,8,bus,11,0.48,14,0,2.75,0.0,0,0,27,26,married
8,9,car,8,0.30,9,0,2.68,0.7,0,0,24,19,single
9,10,car,12,0.43,15,0,2.53,2.1,0,0,26,35,single


### Methodology

The project employs various modeling approaches to predict commuting choices for graduate students, providing insights into factors influencing commuting behavior. This comprehensive approach provides insights into commuting behavior and demonstrates various estimation techniques in data science.

1. **Linear Probability Model (LPM):**
   - **Objective:** Predict commuting choices during winter (driving or taking the bus).
   - **Method:** Utilizes linear regression to estimate the probability of each mode of transport based on cost and time.

2. **Binary Logit Model (BLM):**
   - **Objective:** Explore predictors' relationship with the probability of driving during winter.
   - **Method:** Implements logistic regression to model the binary outcome (driving or taking the bus), offering more precise probability estimates.

3. **Multinomial Logit Model (MLM):**
   - **Objective:** Analyze commuting choices in spring (biking, walking, driving, or taking the bus).
   - **Method:** Applies multinomial logistic regression to estimate probabilities for each commuting mode simultaneously.

4. **Maximum Likelihood Estimation (MLE):**
   - **Objective:** Estimate multinomial logit model parameters for deeper understanding.
   - **Method:** Constructs the likelihood function and maximizes it to estimate parameters, illustrating the theoretical foundations of MLE.

5. **Generalized Method of Moments (GMM):**
   - **Objective:** Estimate binary choice model parameters (driving or taking the bus) using GMM.
   - **Method:** Utilizes moment conditions from the data to estimate parameters, offering a robust estimation approach for potential endogeneity.



### Linear Probability Model (LPM)

The choice to drive to campus during winter is modeled as a linear probability model. The model includes the cost of driving and the time of each alternative as independent variables. The regression equation is defined as follows:

$$
Y_n = \beta_0 + \beta_1C_{nc} + \beta_2T_{nc} + \beta_3T_{nb} + \epsilon_n
$$

where $ Y_n $ is a binary indicator if student $ n $ drives, $ C_{nc} $ is the cost in dollars to student $ n $ of driving to campus, $ T_{nc} $ is the time for student $ n $ to drive, $ T_{nb} $ is the time for student $ n $ to take the bus, and the $ \beta $ coefficients are to be estimated. The OLS regression model is used to estimate the parameters.



In [66]:
#Preprocessing the data
data['mode'].replace({'car':1, 'bus':0}, inplace = True) #need to convert string to int
data.rename(columns = {'time.car': 'driving_time', 'cost.car':'driving_cost', "time.bus":
"bus_time", "cost.bus" :"bus_cost"}, inplace = True) #rename the variables to improve readability
data

,id,mode,driving_time,driving_cost,bus_time,bus_cost,price_gas,snowfall,construction,bus_detour,age,income,marital_status
0,1,1,13,0.79,26,0,2.44,0.0,0,1,33,28,married
1,2,1,9,0.59,14,0,2.59,0.0,0,0,25,28,single
2,3,0,20,1.62,35,0,2.55,1.0,0,0,27,23,single
3,4,0,11,0.30,11,0,2.53,2.2,0,0,25,21,single
4,5,0,12,0.44,15,0,2.49,11.2,0,0,26,26,married
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,0,11,0.40,15,0,2.42,0.0,0,0,27,22,single
996,997,0,9,0.56,16,0,2.42,0.0,0,0,28,24,single
997,998,0,15,0.39,12,0,2.54,1.1,1,0,29,25,single
998,999,0,14,0.32,11,0,2.52,3.6,1,0,28,25,single


In [59]:
#define the formula
formula = 'mode ~ driving_cost + driving_time + bus_time'

#Regress the mode on the variables
model = smf.ols(formula = formula, data = data).fit(cov_type = 'HC1')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   mode   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     64.46
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           4.27e-38
Time:                        15:03:28   Log-Likelihood:                -636.12
No. Observations:                1000   AIC:                             1280.
Df Residuals:                     996   BIC:                             1300.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.9205      0.073     12.627   

The estimated coefficients and heteroskedastic-robust standard errors are reported from this model. These results are briefly interpreted, providing insights into the meaning of each coefficient.

In [29]:
est_coeff = model.params
robust_se = model.HC1_se
print("Coefficients:")
print(est_coeff)
print("\nHeteroskedastic-robust standard errors")
print(robust_se )



Coefficients:
Intercept       0.920511
driving_cost   -0.437564
driving_time   -0.065250
bus_time        0.028367
dtype: float64

Heteroskedastic-robust standard errors
Intercept       0.072901
driving_cost    0.138536
driving_time    0.005831
bus_time        0.006524
dtype: float64


##### Interpretation

beta0: The estimated coefficient for the intercept is 0.9205. It shows predicted probability of choice of drive to campus when all x variables are zero. In this case, there is a significantly high probability (92.05%) of driving even when the cost and time variables are 0.

beta1: The estimated coefficient for cost beta1 is -0.4376. It shows that an increase of $1 in cost of driving decreases probability of driving by 43.76%. 

beta2: The estimated coefficient for driving time beta2 is 0.0653.  It shows  that an increase of 1 minute in driving time decreases probability of driving by 6.53%.

beta3: The estimated coefficient for bus time beta3 is 0.0284. It shows that an increase of 1 minute in bus time increases probability of driving by 2.84%. 

In a linear probability model, one potential issue is that predicted probabilities may fall outside the [0, 1] range. The number of students with infeasible choice probabilities is determined. Based on these results, concern arises regarding the use of a linear probability model in this case.

In [41]:
# Fitted values from the model
data['fitted_values'] = model.fittedvalues

# Q. How many students have infeasible choice probabilities? 
pb_less_zero = (data['fitted_values'] < 0).sum()
pb_more_one =(data['fitted_values'] > 1).sum()
number_of_inf_choice = pb_less_zero + pb_more_one
print("Number of students with infeasible choice probabilities:", number_of_inf_choice)

Number of students with infeasible choice probabilities: 22


The equality in absolute value of the two time coefficients is tested. The interpretation of this test result is provided along with a brief explanation of its intuitive sense. Additionally, the expected impact on the proportion of drivers in case of an equal increase in the time to drive and the time to take the bus is discussed.

Citation: https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLSResults.t_test.html

In [57]:
T_test = model.t_test("-driving_time = bus_time") #plugging in the two time coefficients

t_test_result

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0369      0.009      4.188      0.000       0.020       0.054

P-value < 0.05. So we reject the null hypothesis: two time coefficients are equal in absolute value.

We think the interpreatation of coefficient 0.0369 shows that the absolute value of the coefficient for driving_time is statistically significantly different from the coefficient for bus_time. As coefficient is +ve, we can say, on average, if driving time increases it has a larger effect on probability of driving by car even when there is an equal increase in bus_time.

Intuitively we can say that increase in time to drive has greater effect on the choosing to drive to campus on comparing with an equal increase in time to take the bus. This could be due to the fact that driving is considered more flexible and time-efficient than riding a bus. So people are more sensitive to changes in driving time when they decide their mode of transport.

If a delay were to increase equally the time to drive and the time to take the bus, we would expect the proportion of drivers to decrease. The reason is: greater sensitivity to a change in driving time indicates that individuals are more inclined taking the bus when there is an increase in driving time.

### Binary Logit Model

The model in LPM assumed the probability of driving is a linear function of the data. In reality, however, a different functional form may provide a better fit for the data. 

The choice to drive to campus during winter is modeled as a binary logit model. The cost of driving and the time of each alternative are included as independent variables in the model. The probability $P_n$ that student $n$ drives is represented by the equation:
$$ 
 \ln\left(\frac{P_n}{1 - P_n}\right)= \beta_0 + \beta_1C_{nc} + \beta_2T_{nc} + \beta_3T_{nb}
$$

where $C_{nc}$ is the cost in dollars to student $n$ of driving to campus, $T_{nc}$ is the time for student $n$ to drive, $T_{nb}$ is the time for student $n$ to take the bus, and the $\beta$ coefficients are to be estimated. The binary logit model is used to estimate the parameters.

In [3]:
# Convert the mode to a binary variable where 'car' is 1 and 'bus' is 0
data['mode_bin'] = (data['mode'] == 'car').astype(int)

In [4]:
# Define independent variables
X = data[['cost.car', 'time.car', 'time.bus']]
X = sm.add_constant(X)  

# Define the dependent variable
y = data['mode_bin']

In [5]:
# Fit the binary logit model
model = sm.Logit(y, X)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.600470
         Iterations 6


In [6]:
# Display the summary of the logistic regression model
summary = result.summary()
print(summary)


                           Logit Regression Results                           
Dep. Variable:               mode_bin   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      996
Method:                           MLE   Df Model:                            3
Date:                Mon, 06 May 2024   Pseudo R-squ.:                  0.1205
Time:                        18:39:49   Log-Likelihood:                -600.47
converged:                       True   LL-Null:                       -682.74
Covariance Type:            nonrobust   LLR p-value:                 1.914e-35
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2333      0.347      6.443      0.000       1.554       2.913
cost.car      -2.0772      0.732     -2.836      0.005      -3.513      -0.642
time.car      -0.3322      0.035     -9.400      0.0

The estimated coefficients and standard errors from this model are reported. These results are then interpreted, providing insights into the meaning of each coefficient.

Constant (𝛽0):
- Coefficient: 2.2333
- Standard Error: 0.347
- Interpretation: This represents the log odds of choosing to drive when all independent variables are zero, which isn't practically interpretable in this context.


Cost of driving (β1, for cost.car):
- Coefficient: −2.0772
- Standard Error: 0.732
- Interpretation: This negative coefficient suggests that an increase in the cost of driving decreases the likelihood of choosing to drive. Specifically, for each additional dollar spent on driving, the odds of choosing to drive decrease by a factor of −2.0772


Time to drive (𝛽2, for time.car):
- Coefficient: -0.3322
- Standard Error: 0.035
- Interpretation: This negative coefficient indicates that longer driving times decrease the probability of driving. Each additional minute spent driving reduces the odds of choosing to drive by a factor of -0.3322


Time to take the bus (𝛽3, for time.bus):
- Coefficient: 0.1326
- Standard Error: 0.032
- Interpretation: This positive coefficient suggests that an increase in the time taken to commute by bus slightly increases the likelihood of choosing to drive. Each additional minute on the bus increases the odds of driving by a factor of 0.1326

The marginal effect of each independent variable is calculated for each student, resulting in 3000 marginal effects in total. For each of these three variables, the mean, minimum, maximum, and quartiles of their marginal effects are reported. A comparison is made between these marginal effects and the estimated marginal effects from LPM. The `fittedvalues` function is utilized to calculate the fitted values of a logit model, while the `describe()` function is employed to report these summary statistics for a vector or data frame.

In [7]:
# Calculate the marginal effects
marginal_effects = result.get_margeff()
print(marginal_effects.summary())

# Summarize the marginal effects
marg_effects_frame = marginal_effects.summary_frame()
print(marg_effects_frame.describe())


        Logit Marginal Effects       
Dep. Variable:               mode_bin
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
cost.car      -0.4314      0.150     -2.879      0.004      -0.725      -0.138
time.car      -0.0690      0.006    -11.329      0.000      -0.081      -0.057
time.bus       0.0275      0.007      4.222      0.000       0.015       0.040
          dy/dx  Std. Err.          z      Pr(>|z|)  Conf. Int. Low  \
count  3.000000   3.000000   3.000000  3.000000e+00        3.000000   
mean  -0.157631   0.054158  -3.328704  1.338641e-03       -0.263779   
std    0.241977   0.082882   7.785078  2.297626e-03        0.402414   
min   -0.431426   0.006091 -11.328969  9.430779e-30       -0.725151   
25%   -0.250214   0.006307  -7.103894  1.212468e-05       -0.403045   
50%   -0.069001   0.006522 

In [8]:
marginal_effects = result.get_margeff(at='all')
me_values = marginal_effects.margeff 

me_values_effect_only = me_values

variables = result.model.exog_names[1:] 

marginal_effects_df = pd.DataFrame(me_values, columns=variables)

In [9]:
summary_stats = {}
for var in variables:  
    stats = marginal_effects_df[var].describe()
    summary_stats[var] = {
        'mean': stats['mean'],
        'min': stats['min'],
        '25%': stats['25%'],
        '50%': stats['50%'],
        '75%': stats['75%'],
        'max': stats['max']
    }

print("Marginal Effects Statistics for Selected Variables:")
for var, stats in summary_stats.items():
    print(f"Marginal Effects for '{var}':")
    for stat, value in stats.items():
        print(f"  {stat}: {value:.4f}")
    print()

Marginal Effects Statistics for Selected Variables:
Marginal Effects for 'cost.car':
  mean: -0.4314
  min: -0.5193
  25%: -0.5101
  50%: -0.4772
  75%: -0.3889
  max: -0.0151

Marginal Effects for 'time.car':
  mean: -0.0690
  min: -0.0831
  25%: -0.0816
  50%: -0.0763
  75%: -0.0622
  max: -0.0024

Marginal Effects for 'time.bus':
  mean: 0.0275
  min: 0.0010
  25%: 0.0248
  50%: 0.0305
  75%: 0.0326
  max: 0.0331



The marginal effects derived from the binary logit model reveal how slight variations in each predictor variable (cost and time) influence the likelihood of a student opting to drive to campus. Below are the detailed results:

Marginal Effects

- Driving Costs (cost.car):
- Marginal Effect: -0.4314
- Standard Error: 0.150
- Confidence Interval: [-0.725, -0.138]

This indicates that for every additional dollar spent on driving, the probability of choosing to drive diminishes by approximately 43.14%. This significant effect demonstrates the impact of cost increases on the choice to drive.

- Driving Time (time.car):
- Marginal Effect: -0.0690
- Standard Error: 0.006
- Confidence Interval: [-0.081, -0.057]

Every extra minute driving decreases the likelihood of opting to drive by roughly 6.90%. This statistically significant effect underscores the importance of time efficiency in the driving decision.

- Bus Travel Time (time.bus):
- Marginal Effect: 0.0275
- Standard Error: 0.007
- Confidence Interval: [0.015, 0.040]

In contrast, each additional minute spent on the bus slightly increases the probability of choosing to drive by about 2.75%, indicating that longer bus rides make driving a more appealing option.

The coefficient estimates are utilized to calculate the dollar value attributed by a student to each hour spent driving and each hour spent on the bus. This conversion is achieved by leveraging the coefficient estimates in the model.

In [10]:
# Extract the coefficients for cost and time
beta_cost = result.params['cost.car']
beta_time_car = result.params['time.car']
beta_time_bus = result.params['time.bus']

# Calculate the dollar value per minute
value_per_minute_driving = -beta_time_car / beta_cost
value_per_minute_bus = -beta_time_bus / beta_cost

# Convert to dollar value per hour
value_per_hour_driving = value_per_minute_driving * 60
value_per_hour_bus = value_per_minute_bus * 60

value_per_hour_driving, value_per_hour_bus

(-9.596257191994166, 3.8294944366556596)

Value per hour for driving: Students value each hour spent driving at approximately -$9.60. This negative value indicates a cost or disutility associated with driving time.

Value per hour for taking the bus: Students value each hour spent on the bus at approximately $3.83. This is also a measure of disutility but is less negative than driving, suggesting that students may find bus travel less costly in terms of time compared to driving.

These values are based on the marginal utility of time relative to the cost of driving. Negative values are expected in this context because they represent a cost (or disutility) rather than a benefit.

The choice to drive to campus during winter is modeled as a binary logit model, allowing the parameter on cost to vary inversely with income. The equation representing the probability 
𝑃
𝑛
that student 
𝑛 drives is:

$$ 
 \ln\left(\frac{P_n}{1 - P_n}\right)= \beta_0 + \frac{\beta_1C_nc}{I_n}\ + \beta_2T_nc + \beta_3T_{nb}
$$

Here, 
𝐼
𝑛
represents the income of student 
𝑛. The binary logit model is used to estimate the parameters, incorporating the inverse relationship between cost and income.








In [11]:
data['cost_over_income'] = data['cost.car'] / data['income']
X = data[['cost_over_income', 'time.car', 'time.bus']]
X = sm.add_constant(X)  
y = data['mode_bin'] 

interaction_logit_model = sm.Logit(y, X)
interaction_logit_results = interaction_logit_model.fit()

interaction_logit_results_summary = interaction_logit_results.summary()
interaction_logit_results_summary

Optimization terminated successfully.
         Current function value: 0.597449
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               mode_bin   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      996
Method:                           MLE   Df Model:                            3
Date:                Mon, 06 May 2024   Pseudo R-squ.:                  0.1249
Time:                        18:42:23   Log-Likelihood:                -597.45
converged:                       True   LL-Null:                       -682.74
Covariance Type:            nonrobust   LLR p-value:                 9.498e-37
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                2.2654      0.331      6.842      0.000       1.616       2.914
cost_over_income   -53.6331     14.549     -3.686      0.000     -82.148     -25.118
time.car            -0.3352      0.035     -9.622      0.000      -0.403      -0.267
time.bus             0.1359      0.029      4.719      0.000       0.079       0.192
====================================================================================
"""

The estimated coefficients and standard errors from this model are reported. These results are interpreted to understand the significance of each coefficient. Specifically, the meaning of each coefficient is discussed, providing insight into how changes in the independent variables (cost of driving, time to drive, and time to take the bus) impact the probability of driving to campus during winter.

Constant (β₀) 
- Coefficient: 2.2654

cost_over_income (β1)
- Coefficient: -53.6331
- Standard Error: 14.539
- Z-value: -3.686
- P-value: 0.000
- Interpretation: This coefficient is highly significant and negative, indicating that as the cost of driving relative to income increases, the likelihood of choosing to drive decreases. The size of the coefficient suggests a very strong inverse relationship between the relative cost and the decision to drive, which aligns with the hypothesis that higher economic burden from commuting costs decreases the likelihood of opting to drive.

time.car (β2)
- Coefficient: -0.3352
- Standard Error: 0.035
- Z-value: -9.622
- P-value: 0.000
- Interpretation: Every additional minute spent driving decreases the log-odds of choosing to drive by 0.3352. This suggests that longer driving times make driving less attractive, significantly influencing the decision against driving. The high Z-value and very low P-value indicate strong statistical significance.

time.bus (β3)
- Coefficient: 0.1359
- Standard Error: 0.029
- Z-value: 4.719
- P-value: 0.000
- Interpretation: Every additional minute spent on the bus increases the log-odds of choosing to drive by 0.1359. This might seem counterintuitive initially, but it suggests that as bus travel becomes more time-consuming, driving becomes a relatively more attractive option, possibly due to perceptions of time savings or convenience.

The coefficient estimates are utilized to calculate the marginal utility of income for a student at three different income levels: $15,000, $25,000, and $35,000. For each of these three income levels, the dollar value that a student places on each hour spent driving and on each hour spent on the bus is also calculated.

In [12]:
beta_cost_over_income = -53.6331

average_cost = 1  

income_levels = [15, 25, 35]  

for income in income_levels:
    adjusted_income = income

    # Calculate Marginal Utility of Income
    mu_income = beta_cost_over_income * average_cost / (adjusted_income ** 2)
    
    print(f"Marginal Utility when income ${income * 1000:,}")
    print(f"  {mu_income:.10f}")
    print()


Marginal Utility when income $15,000
  -0.2383693333

Marginal Utility when income $25,000
  -0.0858129600

Marginal Utility when income $35,000
  -0.0437821224



In [13]:
beta_cost_over_income = -53.6331  
beta_time_car = -0.3352
beta_time_bus = 0.1359

for income in [15, 25, 35]:  
    # Calculate the dollar value per hour
    dollar_value_driving = -beta_time_car / (beta_cost_over_income / income) * 60
    dollar_value_bus = -beta_time_bus / (beta_cost_over_income / income) * 60
    
    print(f"When income == ${income * 1000:,}")
    print(f"  Time of drive is {dollar_value_driving:.10f}")
    print(f"  Time of bus is {dollar_value_bus:.10f}")
    print()


When income == $15,000
  Time of drive is -5.6248846328
  Time of bus is 2.2804946945

When income == $25,000
  Time of drive is -9.3748077214
  Time of bus is 3.8008244908

When income == $35,000
  Time of drive is -13.1247308099
  Time of bus is 5.3211542872



### Multinomial Logit Model

The choice of how graduate students commute to campus is being studied once again, but this time, the focus is on this decision during the spring when riding a bike and walking become feasible alternatives. This analysis aims to assist the university in planning for car parking, bike racks, and bus needs during this season. Additionally, the university is considering a modification to bus routes and seeks to understand how this change will impact commute choices.

In [2]:
commute_data = pd.read_csv('commute_multinomial.csv')
display(commute_data)

,id,mode,time.car,cost.car,time.bus,cost.bus,time.bike,cost.bike,time.walk,cost.walk,age,income,marital_status
0,1,bus,16,0.82,20,0,20,0,55,0,33,28,married
1,2,car,10,0.59,16,0,15,0,34,0,25,28,single
2,3,bus,21,1.78,34,0,39,0,105,0,27,23,single
3,4,bus,13,0.31,12,0,12,0,24,0,25,21,single
4,5,bus,11,0.42,16,0,13,0,35,0,26,26,married
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,bus,11,0.39,14,0,14,0,34,0,27,22,single
996,997,bus,11,0.63,14,0,13,0,36,0,28,24,single
997,998,car,8,0.38,10,0,11,0,27,0,29,25,single
998,999,bus,14,0.30,10,0,11,0,19,0,28,25,single


The commute choice during spring is modeled as a multinomial logit model. The representative utility of each alternative is expressed as a linear function of its cost and time. An alternative-specific intercept is included, assuming that cost has a common parameter that does not vary with income, while the parameter on time is allowed to be alternative-specific. 
Specifically, the representative utility $V_{nj}$ to student $n$ from alternative $j$ is represented as:

$$
\ V_{nj} = \alpha_j + \beta_1 C_{nj} + \beta_j T_{nj} \
$$


Here, $V_{nj}$ denotes the representative utility to student $n$ from alternative $j$, $C_{nj}$ represents the cost in dollars to student $n$ of alternative $j$, $T_{nj}$ signifies the time for student $n$ of alternative $j$, and the parameters $\alpha$ and $\beta$ are to be estimated.The MultinomialLogit() function from the xlogit package is utilized to estimate this multinomial logit model. However, it is imperative to first convert the data to a long format before performing the estimation.

In [3]:
# Convert wide format data to long format for analysis
commute_long_format = wide_to_long(commute_data, id_col='id', alt_name='alt', 
                                   varying=['time', 'cost'], 
                                   alt_list=['bike', 'bus', 'car', 'walk'], sep='.', empty_val=0)

# Create time variables for each mode of transportation
commute_long_format['time_bike'] = commute_long_format['time'] * (commute_long_format['alt'] == 'bike')
commute_long_format['time_bus'] = commute_long_format['time'] * (commute_long_format['alt'] == 'bus')
commute_long_format['time_car'] = commute_long_format['time'] * (commute_long_format['alt'] == 'car')
commute_long_format['time_walk'] = commute_long_format['time'] * (commute_long_format['alt'] == 'walk')

# Define the variables to be used in the model
variables = ['cost', 'time_bike', 'time_bus', 'time_car', 'time_walk']

In [4]:
# Initialize and fit the multinomial logistic regression model
multinomial_model = MultinomialLogit()
multinomial_model.fit(X=commute_long_format[variables],
                      y=commute_long_format['mode'],
                      varnames=variables,
                      ids=commute_long_format['id'],
                      alts=commute_long_format['alt'],
                      fit_intercept=True,
                      base_alt='bike')

In [5]:
# Display the model summary
multinomial_model.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 17
    Function evaluations: 19
Estimation time= 0.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
_intercept.bus         -0.2190090     0.3855448    -0.5680506          0.57    
_intercept.car          2.7456687     0.4425950     6.2035691      8.07e-10 ***
_intercept.walk         2.9754379     0.7831786     3.7991817      0.000154 ***
cost                   -2.6044141     0.8235335    -3.1624870       0.00161 ** 
time_bike              -0.2893899     0.0385643    -7.5040963      1.36e-13 ***
time_bus               -0.1431753     0.0351081    -4.0781319       4.9e-05 ***
time_car               -0.4046662     0.0463773    -8.7255208      1.09e-17 ***
time_walk              -0.2966140     0.0384196    -

The estimated coefficients and standard errors from this model are reported, and these results are briefly interpreted. Each coefficient's meaning is discussed, providing insight into the relationship between the predictors (cost and time) and the commuting choices.
1. Car and walking provide more utility than biking (2.75 > 0 and 2.97 > 0) 
2. The intercepts of car and walking are statistically significant
3.  Bus provides the same utility as bike, the parameter is negative but not statistically significant (-0.219 < 0)
4. An additional $1 of cost reduces utility by 2.604
5. An additional 1 min time of bus reduces utility by 0.143
6. An additional 1 min time of car reduces utility by 0.405
7. An additional 1 min time of walk reduces utility by 0.297
8. An additional 1 min time of bike reduces utility by 0.289

The elasticity of each commute alternative with respect to the cost of driving is calculated for each student, resulting in 4000 elasticities (4 alternatives × 1000 students). For each alternative, the mean, minimum, maximum, and quartiles of its elasticity with respect to the cost of driving are reported.

These elasticities and substitution patterns are related to an important property of the logit model. In the logit model, the probabilities of choosing each alternative are determined by their relative utilities, which are influenced by the cost and other characteristics of the alternatives. The elasticities measure the sensitivity of choice probabilities to changes in the cost of driving, providing insights into how changes in cost affect the likelihood of choosing each alternative.

In [6]:
# Display transformed dataset
display(commute_long_format)

,id,alt,time,cost,mode,age,income,marital_status,time_bike,time_bus,time_car,time_walk
0,1,bike,20,0.00,bus,33,28,married,20,0,0,0
1,1,bus,20,0.00,bus,33,28,married,0,20,0,0
2,1,car,16,0.82,bus,33,28,married,0,0,16,0
3,1,walk,55,0.00,bus,33,28,married,0,0,0,55
4,2,bike,15,0.00,car,25,28,single,15,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,999,walk,19,0.00,bus,28,25,single,0,0,0,19
3996,1000,bike,7,0.00,bike,24,34,married,7,0,0,0
3997,1000,bus,11,0.00,bike,24,34,married,0,11,0,0
3998,1000,car,7,0.28,bike,24,34,married,0,0,7,0


In [7]:
# List of predictor variables
predictors = ['cost', 'time_bike', 'time_bus', 'time_car', 'time_walk']

# Predict probabilities using the multinomial logistic regression model
modes, probabilities = multinomial_model.predict(X=commute_long_format[predictors],
                                                 varnames=predictors,
                                                 ids=commute_long_format['id'],
                                                 alts=commute_long_format['alt'],
                                                 return_proba=True)

In [8]:
# Convert probability array to dataframe
probability_df = pd.DataFrame(data=probabilities, columns=['bike', 'bus', 'car', 'walk'])
display(probability_df.head())

,bike,bus,car,walk
0,0.059224,0.885900,0.054845,3.114156e-05
1,0.084750,0.528871,0.381061,5.318120e-03
2,0.002017,0.993032,0.004951,9.387529e-11
3,0.136658,0.634632,0.158824,6.988567e-02
4,0.140002,0.489759,0.366578,3.660774e-03


In [9]:
commute_data

,id,mode,time.car,cost.car,time.bus,cost.bus,time.bike,cost.bike,time.walk,cost.walk,age,income,marital_status
0,1,bus,16,0.82,20,0,20,0,55,0,33,28,married
1,2,car,10,0.59,16,0,15,0,34,0,25,28,single
2,3,bus,21,1.78,34,0,39,0,105,0,27,23,single
3,4,bus,13,0.31,12,0,12,0,24,0,25,21,single
4,5,bus,11,0.42,16,0,13,0,35,0,26,26,married
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,bus,11,0.39,14,0,14,0,34,0,27,22,single
996,997,bus,11,0.63,14,0,13,0,36,0,28,24,single
997,998,car,8,0.38,10,0,11,0,27,0,29,25,single
998,999,bus,14,0.30,10,0,11,0,19,0,28,25,single


In [10]:
# Assign predicted probabilities back to the original DataFrame
commute_data['probability_bike'] = probability_df['bike']
commute_data['probability_bus'] = probability_df['bus']
commute_data['probability_car'] = probability_df['car']
commute_data['probability_walk'] = probability_df['walk']

# Calculate elasticity of cost for each transport mode
commute_data['elasticity_bike'] = -(multinomial_model.coeff_[3]) * commute_data['cost.car'] * commute_data['probability_bike']
commute_data['elasticity_bus'] = -(multinomial_model.coeff_[3]) * commute_data['cost.car'] * commute_data['probability_bus']
commute_data['elasticity_car'] = multinomial_model.coeff_[3] * commute_data['cost.car'] * (1 - commute_data['probability_car'])
commute_data['elasticity_walk'] = -(multinomial_model.coeff_[3]) * commute_data['cost.car'] * commute_data['probability_walk']

In [11]:
display(commute_data)

,id,mode,time.car,cost.car,time.bus,cost.bus,time.bike,cost.bike,time.walk,cost.walk,...,income,marital_status,probability_bike,probability_bus,probability_car,probability_walk,elasticity_bike,elasticity_bus,elasticity_car,elasticity_walk
0,1,bus,16,0.82,20,0,20,0,55,0,...,28,married,0.059224,0.885900,0.054845,3.114156e-05,0.126479,1.891946,-2.018492,6.650652e-05
1,2,car,10,0.59,16,0,15,0,34,0,...,28,single,0.084750,0.528871,0.381061,5.318120e-03,0.130228,0.812665,-0.951064,8.171846e-03
2,3,bus,21,1.78,34,0,39,0,105,0,...,23,single,0.002017,0.993032,0.004951,9.387529e-11,0.009350,4.603556,-4.612906,4.351924e-10
3,4,bus,13,0.31,12,0,12,0,24,0,...,21,single,0.136658,0.634632,0.158824,6.988567e-02,0.110333,0.512382,-0.679139,5.642348e-02
4,5,bus,11,0.42,16,0,13,0,35,0,...,26,married,0.140002,0.489759,0.366578,3.660774e-03,0.153142,0.535724,-0.692871,4.004352e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,bus,11,0.39,14,0,14,0,34,0,...,22,single,0.090500,0.563037,0.342211,4.251921e-03,0.091923,0.571889,-0.668131,4.318768e-03
996,997,bus,11,0.63,14,0,13,0,36,0,...,24,single,0.139027,0.647600,0.210671,2.702213e-03,0.228113,1.062570,-1.295117,4.433739e-03
997,998,car,8,0.38,10,0,11,0,27,0,...,25,single,0.088717,0.410749,0.486582,1.395174e-02,0.087802,0.406509,-0.508118,1.380772e-02
998,999,bus,14,0.30,10,0,11,0,19,0,...,25,single,0.126375,0.585100,0.075306,2.132184e-01,0.098740,0.457153,-0.722486,1.665927e-01


In [28]:
# Calculate and print descriptive statistics for each mode's elasticity
elasticity_bike_stats = commute_data['elasticity_bike'].describe()
elasticity_bus_stats = commute_data['elasticity_bus'].describe()
elasticity_car_stats = commute_data['elasticity_car'].describe()
elasticity_walk_stats = commute_data['elasticity_walk'].describe()

Elasticity statistics for each mode:

In [29]:
elasticity_bike_stats

count    1000.000000
mean        0.103111
std         0.050377
min         0.004748
25%         0.066997
50%         0.094398
75%         0.126483
max         0.360026
Name: elasticity_bike, dtype: float64

- As the cost of driving increases, the probability of biking rises
- Biking is an alternative to driving when the costs of driving escalate.

In [30]:
elasticity_bus_stats

count    1000.000000
mean        0.581732
std         0.586675
min         0.032940
25%         0.198038
50%         0.361621
75%         0.727588
max         4.603556
Name: elasticity_bus, dtype: float64

- As the cost of driving increases, the probability of choosing bus significantly rises.
- Bus is an alternative to driving when the costs of driving escalate.

In [31]:
elasticity_car_stats

count    1000.000000
mean       -0.723662
std         0.564411
min        -4.612906
25%        -0.875862
50%        -0.525610
75%        -0.354964
max        -0.108784
Name: elasticity_car, dtype: float64

- Higher driving costs result in a decreased probability of choose to drive.
- Increasing driving costs encourage individuals to opt for alternatives such as biking or taking the bus

In [32]:
elasticity_walk_stats

count    1.000000e+03
mean     3.881956e-02
std      4.490099e-02
min      4.152576e-12
25%      2.888616e-03
50%      2.026506e-02
75%      6.216582e-02
max      2.656858e-01
Name: elasticity_walk, dtype: float64

- The probability of opting to walk remains unchanged despite rising driving costs
- Walking is not a practical substitute for driving

The model from above is estimated on two subsets of the data based on student marital status. Two separate models are estimated: one using only single students and another using only married students.

In [13]:
# Filter data for married
commute_married = commute_long_format[commute_long_format['marital_status'] == 'married']
commute_married.head(5)

,id,alt,time,cost,mode,age,income,marital_status,time_bike,time_bus,time_car,time_walk
0,1,bike,20,0.00,bus,33,28,married,20,0,0,0
1,1,bus,20,0.00,bus,33,28,married,0,20,0,0
2,1,car,16,0.82,bus,33,28,married,0,0,16,0
3,1,walk,55,0.00,bus,33,28,married,0,0,0,55
16,5,bike,13,0.00,bus,26,26,married,13,0,0,0


In [14]:
# Filter data for single
commute_single = commute_long_format[commute_long_format['marital_status'] == 'single']
commute_single.head(5)

,id,alt,time,cost,mode,age,income,marital_status,time_bike,time_bus,time_car,time_walk
4,2,bike,15,0.00,car,25,28,single,15,0,0,0
5,2,bus,16,0.00,car,25,28,single,0,16,0,0
6,2,car,10,0.59,car,25,28,single,0,0,10,0
7,2,walk,34,0.00,car,25,28,single,0,0,0,34
8,3,bike,39,0.00,bus,27,23,single,39,0,0,0


In [15]:
# Analysis for married
predictors = ['cost', 'time_bike', 'time_bus', 'time_car', 'time_walk']
multinomial_model_married = MultinomialLogit()
multinomial_model_married.fit(X=commute_married[predictors],
                              y=commute_married['mode'],
                              varnames=predictors,
                              ids=commute_married['id'],
                              alts=commute_married['alt'],
                              fit_intercept=True,
                              base_alt='bike')
# Display model summary for married
multinomial_model_married.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 23
    Function evaluations: 26
Estimation time= 0.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
_intercept.bus          0.1545220     0.7989979     0.1933947         0.847    
_intercept.car          4.7810887     0.9873887     4.8421547      1.89e-06 ***
_intercept.walk         4.6099622     2.4525657     1.8796488        0.0609 .  
cost                   -2.7267326     1.4890648    -1.8311713        0.0679 .  
time_bike              -0.3622597     0.0781677    -4.6343911      4.97e-06 ***
time_bus               -0.1821377     0.0589755    -3.0883645       0.00216 ** 
time_car               -0.6565683     0.1008491    -6.5104060      2.44e-10 ***
time_walk              -0.4391819     0.1305149    -

The estimated coefficients and standard errors from both models are reported. These results are then interpreted, providing insights into the meaning of each coefficient and its significance in the context of commuting decisions.

1. Car and walking provides more utility than biking (4.78 and 4.61 > 0)
2. The intercept of car and walking are statistically significant
3. Bus provides the same utility as bike, the parameter is positive but not statistically significant (0.15 > 0)
4. An additional $1 of cost reduces utility by 2.726
5. An additional 1 unit of time reduces utility by 0.18, 0.66, 0.44, 0.36 for bus, car, walk, and bike respectively.

In [16]:
# Analysis for singles
multinomial_model_single = MultinomialLogit()
multinomial_model_single.fit(X=commute_single[predictors],
                             y=commute_single['mode'],
                             varnames=predictors,
                             ids=commute_single['id'],
                             alts=commute_single['alt'],
                             fit_intercept=True,
                             base_alt='bike')
# Display model summary
multinomial_model_single.summary()

Optimization terminated successfully.
    Message: The gradients are close to zero
    Iterations: 16
    Function evaluations: 17
Estimation time= 0.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
_intercept.bus         -0.5526737     0.4516036    -1.2238029         0.221    
_intercept.car          1.9349108     0.4985726     3.8809007      0.000115 ***
_intercept.walk         2.6879916     0.8151522     3.2975336       0.00103 ** 
cost                   -2.7118302     1.0189202    -2.6614746       0.00798 ** 
time_bike              -0.2728774     0.0461655    -5.9108446      5.58e-09 ***
time_bus               -0.1283785     0.0445253    -2.8832678       0.00407 ** 
time_car               -0.3162543     0.0532433    -5.9397916      4.72e-09 ***
time_walk              -0.2695386     0.0395454    -


1. Car and walking provide more utility than biking (1.93 and 2.68 > 0) 
2. The intercepts of car and walking are statistically significant
3.  Bus provides the same utility as bike, the parameter is negative but not statistically significant (-0.553 < 0)
4.  An additional $1 of cost reduces utility by 2.71
5. An additional 1 unit of time reduces utility by 0.13, 0.32, 0.27, 0.27 for bus, car, walk and bike respectively

The sizes of impact of transportation modes and time are larger for married students than for single students

The comparison of the marginal utility of income for single students to that of married students cannot be directly made using the estimated parameters. This limitation arises due to the absence of explicit income coefficients in the models. Without specific modeling of how changes in income affect utility in each group based on their transportation choice, a direct comparison is not feasible based solely on the provided model summaries.

However, it is observed from the cost coefficients that both groups exhibit significant sensitivity to cost changes. This suggests that income considerations inherently influence decision-making through considerations related to cost.

For each marital status, the corresponding parameter estimates are used to calculate the dollar value that a student places on one hour of commute time for each of the four commute alternatives. A direct comparison of these dollar values for single students to those for married students can be made. 

This comparison allows for an understanding of the similarity or difference in these values. It provides insights into how different factors, such as marital status, may influence the valuation of commute time.

The coefficients on commute time indicate the change in utility (or dollar value placed on commute time) associated with a one-hour increase in commute time for each transportation mode.

#### Married

In [17]:
# Extract the coefficients for the model
coefficients_married = multinomial_model_married.coeff_

# Calculate the value of time for each mode of transport
value_of_time_car_married = (-coefficients_married[6] / coefficients_married[3]) * 60  
value_of_time_bus_married = (-coefficients_married[5] / coefficients_married[3]) * 60  
value_of_time_bike_married = (-coefficients_married[4] / coefficients_married[3]) * 60  
value_of_time_walk_married = (-coefficients_married[7] / coefficients_married[3]) * 60   

value_of_time_car_married, value_of_time_bus_married, value_of_time_bike_married, value_of_time_walk_married

(-14.447364220310995,
 -4.0078236201909405,
 -7.971291459013427,
 -9.663915649815012)

- The dollar value that a student places on 1 hour driving: 14.45
- The dollar value that a student places on 1 hour bus:  4.01
- The dollar value that a student places on 1 hour biking: 7.97
- The dollar value that a student places on 1 hour walking: 9.66

##### Singles

In [18]:
# Extract the coefficients for the model
coefficients_single = multinomial_model_single.coeff_

# Calculate the value of time for each mode of transport
value_of_time_car_single = (-coefficients_single[6] / coefficients_single[3]) * 60  
value_of_time_bus_single = (-coefficients_single[5] / coefficients_single[3]) * 60  
value_of_time_bike_single = (-coefficients_single[4] / coefficients_single[3]) * 60  
value_of_time_walk_single = (-coefficients_single[7] / coefficients_single[3]) * 60

value_of_time_car_single, value_of_time_bus_single, value_of_time_bike_single, value_of_time_walk_single

(-6.997215613888858,
 -2.8404102113682943,
 -6.037488159073421,
 -5.963615962715602)

- The dollar value that a student places on 1 hour driving: 6.997
- The dollar value that a student places on 1 hour bus:  2.84
- The dollar value that a student places on 1 hour biking: 6.04
- The dollar value that a student places on 1 hour walking: 5.96

##### Comparison

We can compare these dollar values for single students to those for married students. If we compare the two groups of dollar values between martial status, we can see that:
1. Car: Married students (14.45) have a significantly higher value on one hour of car commute time compared to single students (6.997).
2. Bus: The dollar value of one hour of commute time for married students (4.01) is higher (i.e., they value their time more) compared to single students (2.84).
4. Biking: Similarly, married students (7.977) value one hour of biking commute time more than single students (6.04).
3. Walking: The value of one hour of walking commute time is also higher for married students (9.66) compared to single students (5.96)



These dollars values on each of the four commuting options are generally higher for the married students than the single students, especially when it comes to driving cars. Also, students generally place the highest value on their time when driving to school and the lowest when taking buses.

Now, say the university's commitment to environmental sustainability has led to a proposal aimed at encouraging graduate students to take the bus rather than drive to campus. One proposed solution involves introducing more buses on the existing bus routes, thereby reducing bus commute time by 20%.The parameter estimates from part above are utilized to simulate this counterfactual scenario, allowing for an assessment of the potential impact of the proposed intervention on commuting choices.

The reduction in bus commute time is expected to influence the commuting choices of additional students among the 1000 students in this dataset. Specifically, it is anticipated that more students will opt to commute by bus due to this reduction in bus commute time. 

Conversely, fewer students are expected to choose each of the three other commute alternatives as a result of this change. This analysis provides insights into the potential shifts in commuting behavior resulting from changes in bus commute time.

In [3]:
# Create a modified dataset where bus travel time is reduced by 20%
commute_data_adjusted_bus = commute_data.copy()
commute_data_adjusted_bus['time.bus'] *= 0.8

In [20]:
# Convert the adjusted dataset from wide to long format
commute_long_adjusted_bus = wide_to_long(commute_data_adjusted_bus, id_col='id', alt_name='alt', 
                                         varying=['time', 'cost'], 
                                         alt_list=['bike', 'bus', 'car', 'walk'], sep='.', empty_val=0)
display(commute_long_adjusted_bus.head(5))

,id,alt,time,cost,mode,age,income,marital_status,probability_bike,probability_bus,probability_car,probability_walk,elasticity_bike,elasticity_bus,elasticity_car,elasticity_walk
0,1,bike,20.0,0.00,bus,33,28,married,0.059224,0.885900,0.054845,0.000031,0.126479,1.891946,-2.018492,0.000067
1,1,bus,16.0,0.00,bus,33,28,married,0.059224,0.885900,0.054845,0.000031,0.126479,1.891946,-2.018492,0.000067
2,1,car,16.0,0.82,bus,33,28,married,0.059224,0.885900,0.054845,0.000031,0.126479,1.891946,-2.018492,0.000067
3,1,walk,55.0,0.00,bus,33,28,married,0.059224,0.885900,0.054845,0.000031,0.126479,1.891946,-2.018492,0.000067
4,2,bike,15.0,0.00,car,25,28,single,0.084750,0.528871,0.381061,0.005318,0.130228,0.812665,-0.951064,0.008172


In [21]:
# Calculate time variables for each transportation mode in the adjusted dataset
commute_long_adjusted_bus['time_bike'] = commute_long_adjusted_bus['time'] * (commute_long_adjusted_bus['alt'] == 'bike')
commute_long_adjusted_bus['time_bus'] = commute_long_adjusted_bus['time'] * (commute_long_adjusted_bus['alt'] == 'bus')
commute_long_adjusted_bus['time_car'] = commute_long_adjusted_bus['time'] * (commute_long_adjusted_bus['alt'] == 'car')
commute_long_adjusted_bus['time_walk'] = commute_long_adjusted_bus['time'] * (commute_long_adjusted_bus['alt'] == 'walk')

# Define variables for model prediction
variables = ['cost', 'time_bike', 'time_bus', 'time_car', 'time_walk']

# Predict transportation mode probabilities for the original data
alter_original, prob_original = multinomial_model.predict(X=commute_long_format[variables],
                                              varnames=variables,
                                              ids=commute_long_format['id'],
                                              alts=commute_long_format['alt'],
                                              return_proba=True)
# Aggregate the probabilities
aggregate_original = pd.DataFrame(data=prob_original, columns=['bike', 'bus', 'car', 'walk']).sum()

In [22]:
# Predict transportation mode probabilities for the adjusted bus time data
alter_adjusted, prob_adjusted = multinomial_model.predict(X=commute_long_adjusted_bus[variables],
                                              varnames=variables,
                                              ids=commute_long_adjusted_bus['id'],
                                              alts=commute_long_adjusted_bus['alt'],
                                              return_proba=True)

# Aggregate the probabilities
aggregate_adjusted_bus = pd.DataFrame(data=prob_adjusted, columns=['bike', 'bus', 'car', 'walk']).sum()

# Calculate and print the differences in aggregated probabilities
difference = aggregate_adjusted_bus - aggregate_original
print(difference)

bike   -17.646702
bus     78.164974
car    -54.675649
walk    -5.842624
dtype: float64


In [23]:
print(difference / aggregate_original)  # Relative change in probabilities

bike   -0.156166
bus     0.172550
car    -0.145802
walk   -0.099027
dtype: float64


- There were 453 bus users at first and because of the reduction in bus commute time, this number is expected to increase by 78 students.
- 18 fewer students using bike, 55 fewer students drive car, 6 fewer walk.

##### The reduction in bus commute time is expected to generate additional economic surplus for the 1000 students in this dataset.

In [25]:
# Extract coefficients from the model
coefficients = multinomial_model.coeff_

# Calculate the utility (v_nj) for each alternative in the original dataset
commute_long_format['v_nj'] = (
    (coefficients[0] + coefficients[3] * commute_long_format['cost'] + coefficients[5] * commute_long_format['time_bus']) * (commute_long_format['alt'] == 'bus') +
    (coefficients[1] + coefficients[3] * commute_long_format['cost'] + coefficients[6] * commute_long_format['time_car']) * (commute_long_format['alt'] == 'car') +
    (coefficients[2] + coefficients[3] * commute_long_format['cost'] + coefficients[7] * commute_long_format['time_walk']) * (commute_long_format['alt'] == 'walk') +
    (coefficients[3] * commute_long_format['cost'] + coefficients[4] * commute_long_format['time_bike']) * (commute_long_format['alt'] == 'bike')
)
commute_long_format['v_nj'] = np.exp(commute_long_format['v_nj'])

# Apply the same calculations for the adjusted bus time dataset
commute_long_adjusted_bus['v_nj'] = (
    (coefficients[0] + coefficients[3] * commute_long_adjusted_bus['cost'] + coefficients[5] * commute_long_adjusted_bus['time_bus']) * (commute_long_adjusted_bus['alt'] == 'bus') +
    (coefficients[1] + coefficients[3] * commute_long_adjusted_bus['cost'] + coefficients[6] * commute_long_adjusted_bus['time_car']) * (commute_long_adjusted_bus['alt'] == 'car') +
    (coefficients[2] + coefficients[3] * commute_long_adjusted_bus['cost'] + coefficients[7] * commute_long_adjusted_bus['time_walk']) * (commute_long_adjusted_bus['alt'] == 'walk') +
    (coefficients[3] * commute_long_adjusted_bus['cost'] + coefficients[4] * commute_long_adjusted_bus['time_bike']) * (commute_long_adjusted_bus['alt'] == 'bike')
)
commute_long_adjusted_bus['v_nj'] = np.exp(commute_long_adjusted_bus['v_nj'])

# Calculate the log-sum of utilities for both datasets and their difference
df1 = commute_long_format.groupby(['id'])['v_nj'].sum().reset_index()
df1['log_sum_v'] = np.log(df1['v_nj'])

df2 = commute_long_adjusted_bus.groupby(['id'])['v_nj'].sum().reset_index()
df2['log_sum_v'] = np.log(df2['v_nj'])

In [26]:
# Calculate the change in log-sum due to the reduced bus time
df3 = df1
df3['change_in_log_sum'] = (df2['log_sum_v'] - df1['log_sum_v']) / (-coefficients[3])

In [27]:
# Sum up the changes and print
total_change_in_log_sum = df3['change_in_log_sum'].sum()
print(total_change_in_log_sum)

82.74060812045207


Economic surplus is expected to increase by 82.74 because of this reduction on bus commute time.

In [1]:
import statsmodels.api as sm
import statsmodels as sms
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import xlogit
from xlogit.utils import wide_to_long
from xlogit import MixedLogit
from xlogit import MultinomialLogit
import scipy
from scipy.optimize import minimize
from scipy.stats import norm
import numpy.linalg as lin

### Maximum Likelihood Estimation(MLE)

The model is being estimated "by hand" to better understand the maximum likelihood estimation method for how graduate students in town X choose how to commute to campus in the spring when riding a bike and walking are feasible alternatives.

The commute choice during spring is modeled as a multinomial logit model. The representative utility of each alternative is expressed as a linear function of its cost and time with common parameters on these variables. Specifically, the representative utility $V_{nj}$ to student $ n $ from alternative $j$ is represented as:
$$
 V_{nj} = \beta C_{nj} + \gamma T_{nj} \
$$

Here, $C_{nj}$ denotes the cost to student $n$ of alternative $ j $, $T_{nj} $ signifies the time for student $n$ of alternative $ j $, and the parameters $ \beta$ and $\gamma$ are to be estimated. These parameters are estimated by maximum likelihood estimation.


In [3]:
data = pd.read_csv("commute_multinomial.csv")
data

,id,mode,time.car,cost.car,time.bus,cost.bus,time.bike,cost.bike,time.walk,cost.walk,age,income,marital_status
0,1,bus,16,0.82,20,0,20,0,55,0,33,28,married
1,2,car,10,0.59,16,0,15,0,34,0,25,28,single
2,3,bus,21,1.78,34,0,39,0,105,0,27,23,single
3,4,bus,13,0.31,12,0,12,0,24,0,25,21,single
4,5,bus,11,0.42,16,0,13,0,35,0,26,26,married
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,bus,11,0.39,14,0,14,0,34,0,27,22,single
996,997,bus,11,0.63,14,0,13,0,36,0,28,24,single
997,998,car,8,0.38,10,0,11,0,27,0,29,25,single
998,999,bus,14,0.30,10,0,11,0,19,0,28,25,single


In [4]:
def function(params, data): # creating function that takes a set of parameters and data as inputs: function(parameters, data).
    beta1 = params[0] # name the parameters
    beta2 = params[1]
#i. Calculate the representative utility of every alternative for each decision maker
    data = data.copy()
    data['utility_car'] = beta1 * data['cost.car'] + beta2 * data['time.car']
    data['utility_bus'] = beta1 * data['cost.bus'] + beta2 * data['time.bus']
    data['utility_bike'] = beta1 * data['cost.bike'] + beta2 * data['time.bike']
    data['utility_walk'] = beta1 * data['cost.walk'] + beta2 * data['time.walk']
    
#logit choice probability denominator given the parameter
    data['prob_denom'] = np.exp(data['utility_car']) + np.exp(data['utility_bus']) +\
    np.exp(data['utility_bike']) + np.exp(data['utility_walk'])
    
# logit choice probability for each alternative given the parameter
    data['prob_car'] = np.exp(data['utility_car']) / data['prob_denom']
    data['prob_bus'] = np.exp(data['utility_bus']) / data['prob_denom']
    data['prob_bike'] = np.exp(data['utility_bike']) / data['prob_denom']
    data['prob_walk'] = np.exp(data['utility_walk']) / data['prob_denom']
    
#ii. Calculate the choice probability of the chosen alternative for each decision maker
    data['prob_choice'] = data['prob_car'] * (data['mode'] == 'car') +\
    data['prob_bus'] * (data['mode'] == 'bus') + data['prob_bike'] * (data['mode'] == 'bike') +\
    data['prob_walk'] * (data['mode'] == 'walk')
    
# log of logit choice probability for chosen alternative given the parameter
    data['log_prob'] = np.log(data['prob_choice'])
    
#iii. Sum the log of these choice probabilities to get the log-likelihood.
    log_likelihood = data['log_prob'].sum()
    
# (iv). Return the negative of the log-likelihood.
    return -log_likelihood

The log-likelihood is maximized (or its negative is minimized) using the minimize() function. Parameter estimates, standard errors, z-stats, and p-values are reported. These results are then interpreted, explaining the meaning of each parameter.

In [5]:
#Maximize the log-likelihood (by minimizing its negative) using minimize()

initial_parameters = [0]*2 # we have beta1 and beta2 from above
model = minimize(function, initial_parameters, args = (data), method = 'BFGS', options = {'disp' : True})
print("\nThe result of the optimization:", model) #optimization results

Optimization terminated successfully.
         Current function value: 1219.844944
         Iterations: 12
         Function evaluations: 90
         Gradient evaluations: 30

The result of the optimization:   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1219.8449442611861
        x: [-1.002e+00 -1.257e-01]
      nit: 12
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 1.866e-02  4.044e-04]
            [ 4.044e-04  6.009e-05]]
     nfev: 90
     njev: 30


In [6]:
print("The parameter estimates are", model.x) 
print("The standard errors are", np.sqrt(np.diag(model.hess_inv))) 
print("The z-stats are", model.x/np.sqrt(np.diag(model.hess_inv))) 
print("The p-values are", 2*norm.cdf(-abs(model.x/np.sqrt(np.diag(model.hess_inv)))))

The parameter estimates are [-1.00172165 -0.12565614]
The standard errors are [0.13661193 0.00775171]
The z-stats are [ -7.33260714 -16.21012185]
The p-values are [2.25717893e-13 4.27719769e-59]


From above we get the following parameter values (beta1, beta2 respectively): 
-1.00172165 and -0.12565614.

For beta parameters, the interpretation is that utility will fall by -
- 1.00172165 if cost of mode of transport goes up by $1
- 0.12565614 if time taken by mode of transport goes up by a minute


Standard errors 0.13661193, 0.00775171 for beta1 and beta2 are small which means that the estimates are likely close to the true popultion value.

Z stats are -7.33260714, -16.21012185  in absolute terms show that the parameter estimates significantly differ from zero.

As P-values  2.25717893e-13, 4.27719769e-59 < 0.05 we reject the null hypothesis. Both beta1 an beta2 are statistically significant and affect the utility of chosen mode of commute.


The commute choice during spring is modeled again as a multinomial logit model, but with the addition of alternative-specific intercepts for all but one alternative. The representative utility to student $n$ from alternative $j$ is expressed as:
 $$
 Vnj = αj + βCnj + γTnj
 $$ 
  Here, $C_nj$ is the cost to student $n$ of alternative $j$, $Tnj$ is the time for student $n$ of alternative $j$, and the $α$, $β$, and $γ$ parameters are to be estimated. he parameters of this model are estimated by maximum likelihood estimation. The steps to creating the maximum likelihood estimator are the same as above, but some of the calculations will differ.

In [7]:
def alt_function(params, data): # creating function that takes a set of parameters and data as inputs: function(parameters, data).
    a_car = params[0] #name the parameters
    a_bus = params[1]
    a_walk = params[2]
    beta1 = params[3]
    beta2 = params[4]
#i. Calculate the representative utility of every alternative for each decision maker
    data = data.copy()
    data['utility_car'] = a_car + beta1 * data['cost.car'] + beta2 * data['time.car']
    data['utility_bus'] = a_bus + beta1 * data['cost.bus'] + beta2 * data['time.bus']
    data['utility_walk'] = a_walk + beta1 * data['cost.walk'] + beta2 * data['time.walk']
    data['utility_bike'] = beta1 * data['cost.bike'] + beta2 * data['time.bike']
    
#logit choice probability denominator given the parameter
    data['prob_denom'] = np.exp(data['utility_car']) + np.exp(data['utility_bus']) +\
    np.exp(data['utility_bike']) + np.exp(data['utility_walk'])
    
# logit choice probability for each alternative given the parameter
    data['prob_car'] = np.exp(data['utility_car']) / data['prob_denom']
    data['prob_bus'] = np.exp(data['utility_bus']) / data['prob_denom']
    data['prob_bike'] = np.exp(data['utility_bike']) / data['prob_denom']
    data['prob_walk'] = np.exp(data['utility_walk']) / data['prob_denom']
    
#ii. Calculate the choice probability of the chosen alternative for each decision maker
    data['prob_choice'] = data['prob_car'] * (data['mode'] == 'car') +\
    data['prob_bus'] * (data['mode'] == 'bus') + data['prob_bike'] * (data['mode'] == 'bike') +\
    data['prob_walk'] * (data['mode'] == 'walk')
    
# log of logit choice probability for chosen alternative given the parameter
    data['log_prob'] = np.log(data['prob_choice'])
    
#iii. Sum the log of these choice probabilities to get the log-likelihood.
    log_likelihood = data['log_prob'].sum()
    
# (iv). Return the negative of the log-likelihood.
    return -log_likelihood

In [8]:
#Similarly, maximize the log-likelihood (by minimizing its negative) using minimize()

initial_parameters = [0]*5 # we have 5 parameters from above
model_alt = minimize(alt_function, initial_parameters, args = (data), method = 'BFGS', tol = 10e-3, options = {'disp' : True}) #changed tolerance level
print("\nThe result of the optimization:", model) #optimization results

Optimization terminated successfully.
         Current function value: 1005.448759
         Iterations: 15
         Function evaluations: 108
         Gradient evaluations: 18

The result of the optimization:   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1219.8449442611861
        x: [-1.002e+00 -1.257e-01]
      nit: 12
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 1.866e-02  4.044e-04]
            [ 4.044e-04  6.009e-05]]
     nfev: 90
     njev: 30


The parameter estimates, standard errors, z-stats, and p-values are reported, and the results are interpreted accordingly. Each parameter's meaning is discussed in relation to the model's context and its impact on the commuting choices of students.

In [10]:
print("The parameter estimates are", model_alt.x) 
print("The standard errors are", np.sqrt(np.diag(model_alt.hess_inv))) 
print("The z-stats are", model_alt.x/np.sqrt(np.diag(model_alt.hess_inv))) 
print("The p-values are", 2*norm.cdf(-abs(model_alt.x/np.sqrt(np.diag(model_alt.hess_inv)))))

The parameter estimates are [ 2.9247626   1.76032516  3.1718253  -6.05465741 -0.29576254]
The standard errors are [0.20225211 0.11504937 0.30580191 0.51212601 0.0237813 ]
The z-stats are [ 14.46097441  15.30060636  10.37215647 -11.82259309 -12.43677132]
The p-values are [2.13750204e-47 7.57476032e-53 3.31947787e-25 2.98331420e-32
 1.65041217e-35]


From above we get the following parameter values (a_car, a_bus, a_walk, beta1, beta2 respectively):
2.9247626,1.76032516,3.1718253,-6.05465741 and -0.29576254.
 
For alpha paramters, the interpretation is that utility will go up by  -
- 2.92476163 if car is chosen as mode of transport
- 1.76032508 if bus is chosen as mode of transport
- 3.17182458 if walking is chosen as mode of transport
- Bike does not have an alpha intercept as we chose this as the reference category. So utilities of all other modes are relative to bike

For beta parameters, the interpretation is that utility will fall by -
- 6.05465741 if cost of mode of transport goes up by $1
- 0.29576254 if time taken by mode of transport goes up by a minute



Standard errors 0.20225211 0.11504937 0.30580191 0.51212601 0.0237813  are small which means that the estimates are likely close to the true population value.

Z stats are 14.46097441  15.30060636  10.37215647 -11.82259309 -12.43677132  in absolute terms show that the parameter estimates significantly differ from zero.

As P-values  2.13750204e-47 7.57476032e-53 3.31947787e-25 2.98331420e-32
 1.65041217e-35 < 0.05 we reject the null hypothesis. All parameters are statistically significant and affect the utility of chosen mode of commute.


A likelihood ratio test is conducted on this model to examine the joint significance of the alternative-specific intercepts. The null hypothesis being tested is: 

$$ H_0: \alpha_{\text{bus}} = \alpha_{\text{car}} = \alpha_{\text{walk}} = 0 \$$

The specific formulation of the null hypothesis may vary depending on the chosen reference alternative. 

The result of the test determines whether to reject the null hypothesis. The p-value of the test indicates the significance level of the observed differences between the models. 

In this case, the interpretation of the test result is provided based on whether the null hypothesis is rejected and the associated p-value.

In [12]:
#negative of log-likelihood value for each model
print(model.fun, model_alt.fun)
print(2 * (- model_alt.fun - (- model.fun))) #likelihood ratio test statistic
from scipy.stats import chi2
chi2.ppf(0.95, 3) #chi-squared critical value , dof 3 as we estimate 3 parameters

1219.8449442611861 1005.4487590830483
428.79237035627557


7.814727903251179

In [13]:
#Is likelihood ratio test statistic > critical value ?
print(2 * (- model_alt.fun - (- model.fun)) > chi2.ppf(0.95, 5))

print(1 - chi2.cdf(2 * (- model_alt.fun - (- model.fun)), 5)) # p-value 

True
0.0


We see that the test statistic is greater than the critical value and the p-value is < 0.05. We reject the null hypothesis H0: αbus = αcar = αwalk = 0. The interpretation is that levels of utility vary with different modes of transport so an individual will prefer that mode which gives highest utility.

In [16]:
#The commute choice during spring is once again modeled as a multinomial logit model,
#but with the parameter on time allowed to be alternative-specific. 

In [17]:
def function_alt(params, data): # creating function that takes a set of parameters and data as inputs: function(parameters, data).
    a_car = params[0] #name the parameters
    a_bus = params[1]
    a_walk = params[2]
    beta1 = params[3]
    beta2 = params[4] #car
    beta3 = params[5] #bus
    beta4 = params[6] #bike
    beta5 = params[7] #walk
    
#i. Calculate the representative utility of every alternative for each decision maker
    data = data.copy()
    data['utility_car'] = a_car + beta1 * data['cost.car'] + beta2 * data['time.car']
    data['utility_bus'] = a_bus + beta1 * data['cost.bus'] + beta3 * data['time.bus']
    data['utility_walk'] = a_walk + beta1 * data['cost.walk'] + beta5 * data['time.walk']
    data['utility_bike'] = beta1 * data['cost.bike'] + beta4 * data['time.bike']
    
#logit choice probability denominator given the parameter
    data['prob_denom'] = np.exp(data['utility_car']) + np.exp(data['utility_bus']) +\
    np.exp(data['utility_bike']) + np.exp(data['utility_walk'])
    
# logit choice probability for each alternative given the parameter
    data['prob_car'] = np.exp(data['utility_car']) / data['prob_denom']
    data['prob_bus'] = np.exp(data['utility_bus']) / data['prob_denom']
    data['prob_bike'] = np.exp(data['utility_bike']) / data['prob_denom']
    data['prob_walk'] = np.exp(data['utility_walk']) / data['prob_denom']
    
#ii. Calculate the choice probability of the chosen alternative for each decision maker
    data['prob_choice'] = data['prob_car'] * (data['mode'] == 'car') +\
    data['prob_bus'] * (data['mode'] == 'bus') + data['prob_bike'] * (data['mode'] == 'bike') +\
    data['prob_walk'] * (data['mode'] == 'walk')
    
# log of logit choice probability for chosen alternative given the parameter
    data['log_prob'] = np.log(data['prob_choice'])
    
#iii. Sum the log of these choice probabilities to get the log-likelihood.
    log_likelihood = data['log_prob'].sum()
    
# (iv). Return the negative of the log-likelihood.
    return -log_likelihood

In [18]:
#Again, maximize the log-likelihood (by minimizing its negative) using minimize()

initial_parameters = [0]*8 # we have 8 parameters from above
model_time = minimize(function_alt, initial_parameters, args = (data), method = 'BFGS', tol = 10e-3, options = {'disp' : True}) #changed tolerance level
print("\nThe result of the optimization:", model) #optimization results

Optimization terminated successfully.
         Current function value: 982.356064
         Iterations: 19
         Function evaluations: 207
         Gradient evaluations: 23

The result of the optimization:   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1219.8449442611861
        x: [-1.002e+00 -1.257e-01]
      nit: 12
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 1.866e-02  4.044e-04]
            [ 4.044e-04  6.009e-05]]
     nfev: 90
     njev: 30


The parameter estimates, standard errors, z-stats, and p-values are reported, and the results are interpreted accordingly. Each parameter's meaning is discussed in relation to the model's context and its impact on the commuting choices of students.

In [20]:
print("The parameter estimates are", model_time.x) 
print("The standard errors are", np.sqrt(np.diag(model_time.hess_inv))) 
print("The z-stats are", model_time.x/np.sqrt(np.diag(model_time.hess_inv))) 
print("The p-values are", 2*norm.cdf(-abs(model_time.x/np.sqrt(np.diag(model_time.hess_inv)))))

The parameter estimates are [ 2.74569037 -0.21900146  2.97546931 -2.60438182 -0.40466758 -0.14317438
 -0.28938822 -0.29661517]
The standard errors are [0.45733633 0.40108321 0.80065119 0.85931873 0.04563565 0.03590145
 0.03891019 0.03894512]
The z-stats are [ 6.00365679 -0.546025    3.71631162 -3.03075185 -8.86735643 -3.98798318
 -7.43733814 -7.6162347 ]
The p-values are [1.92922288e-09 5.85048752e-01 2.02152237e-04 2.43945650e-03
 7.49028571e-19 6.66373857e-05 1.02734338e-13 2.61182592e-14]


From above we get the following parameter values (a_car, a_bus, a_walk, beta1, beta2, beta3, beta4, beta5 respectively):
2.74569037, -0.21900146, 2.97546931, -2.60438182, -0.40466758, -0.14317438, -0.28938822, -0.29661517 


For alpha paramters, the interpretation is that -
- Utility will go up by 2.74569037 if car is chosen as mode of transport
- Utility will fall by 0.21900146 if bus is chosen as mode of transport
- Utility will go up by 2.97546931 if walking is chosen as mode of transport
- Bike does not have an alpha intercept as we chose this as the reference category. So utilities of all other modes are relative to bike

For beta parameters, the interpretation is that utility will fall by -
- 2.60438182 if cost of mode of transport goes up by $1
- 0.40466758 if time taken by car goes up by a minute
- 0.14317438 if time taken by bus goes up by a minute
- 0.28938822 if time taken by bike goes up by a minute
- 0.29661517 if time taken by walking goes up by a minute

Standard errors 0.45733633 0.40108321 0.80065119 0.85931873 0.04563565 0.03590145 0.03891019 0.03894512 are small which means that the estimates are likely close to the true popultion value.

Z stats are 16.00365679 -0.546025    3.71631162 -3.03075185 -8.86735643 -3.98798318  -7.43733814 -7.6162347 in absolute terms show that the parameter estimates significantly differ from zero.

As P-values 1.92922288e-09 5.85048752e-01 2.02152237e-04 2.43945650e-03 7.49028571e-19 6.66373857e-05 1.02734338e-13 2.61182592e-14 < 0.05 we reject the null hypothesis. All parameters are statistically significant and affect the utility of chosen mode of commute.

In [22]:
#negative of log-likelihood value for each model
print(model_alt.fun, model_time.fun)
print(2 * (- model_time.fun - (- model_alt.fun))) #likelihood ratio test statistic
from scipy.stats import chi2
chi2.ppf(0.95, 4) #chi-squared critical value , dof 4 as we estimate 4 parameters

1005.4487590830483 982.3560638353069
46.18539049548281


9.487729036781154

In [23]:
#Is likelihood ratio test statistic > critical value ?
print(2 * (- model_time.fun - (- model_alt.fun)) > chi2.ppf(0.95, 4))

print(1 - chi2.cdf(2 * (- model_time.fun - (- model_alt.fun)), 4)) # p-value 

True
2.253488062819997e-09


We see that the test statistic is greater than the critical value and the p-value is < 0.05. We reject the null hypothesis H0: γbike = γbus = γcar = γwalk. The interpretation is that levels of utility vary with difference in the time taken by different modes of transport so an individual will prefer that mode which gives highest utility.

### Generalized Method of Moments(GMM)

The choice to drive to campus during winter is modeled as a binary logit model. The representative utility of each alternative is expressed as a linear function of its cost and time. An alternative-specific intercept is included, and the parameter on time is allowed to be alternative-specific. Specifically, the representative utility to student 
𝑛
n from driving and taking the bus, respectively, are represented as:
$$
Vnc = α + βCnc + γcarTnc
$$
$$
Vnb = γbusTnb
$$

Here, $ C_{nj} $ represents the cost to student $n$ of alternative $j$, $T_{nj} $ signifies the time for student $n$ of alternative \$j$, and the $ \alpha $, $ \beta $, and $ \gamma $ parameters are to be estimated. The model excludes a bus-specific intercept term because only one intercept term is identified in this model, and the bus cost is excluded because it is free for all students. 

The difference in representative utility between driving and taking the bus for student $n$ is given by:

$$ V_{nc} - V_{nb} = \alpha + \beta C_{nc} + \gamma_{\text{car}} T_{nc} - \gamma_{\text{bus}} T_{nb} $$

Because this is a binary logit model, the choice probability of driving is expressed as a function of $ V_{nc} - V_{nb} $ as:

$$ P_{nc} = \frac{1}{1 + e^{-(V_{nc} - V_{nb})}} $$

The parameters of this model are estimated by the method of moments.

Four moment conditions are formulated for this model.



![image-2.png](attachment:image-2.png)

In [25]:
# Load the binary dataset
df_bi = pd.read_csv("commute_binary.csv")
df_bi["mode"].replace({'car' : 1, 'bus' : 0}, inplace=True)
df_bi.head()

,id,mode,time.car,cost.car,time.bus,cost.bus,price_gas,snowfall,construction,bus_detour,age,income,marital_status
0,1,1,13,0.79,26,0,2.44,0.0,0,1,33,28,married
1,2,1,9,0.59,14,0,2.59,0.0,0,0,25,28,single
2,3,0,20,1.62,35,0,2.55,1.0,0,0,27,23,single
3,4,0,11,0.30,11,0,2.53,2.2,0,0,25,21,single
4,5,0,12,0.44,15,0,2.49,11.2,0,0,26,26,married


In [26]:
# Create dataset for use in MM moment function
mm_data = df_bi.assign(constant=1).loc[:, ['mode',
                                           'constant',
                                           'cost.car',
                                           'time.car',
                                           'time.bus']
                                      ].to_numpy()

In [27]:
print(mm_data)

[[ 1.    1.    0.79 13.   26.  ]
 [ 1.    1.    0.59  9.   14.  ]
 [ 0.    1.    1.62 20.   35.  ]
 ...
 [ 0.    1.    0.39 15.   12.  ]
 [ 0.    1.    0.32 14.   11.  ]
 [ 1.    1.    0.28  9.   10.  ]]


 A function is created that takes a set of parameters and a matrix of data as inputs: function(parameters, data matrix).


In [28]:
### Gradient function
def mm_fn(params, data_matrix):
    # Select data for X [N x K]
    X = data_matrix[:, 1:]
    # Select data for y [N x 1]
    y = data_matrix[:, 0]
    # (i).Calculate the difference in representative utility for each decision maker. [N x 1]
    utility = np.dot(X, params)
    # (ii).Calculate logit choice probability for each decision maker. [N x 1]
    prob = 1 / (1 + np.exp(-utility))
    # (iii).Calculate econometric residuals [N x 1]
    residuals = y - prob
    # (iv).Calculate each of the 4 moments for each decision maker.
    moments = np.multiply(residuals.reshape(-1, 1), X)
    # (v).Return the N × L matrix of individual moments.
    return moments

In [29]:
# Define objective function for GMM
def gmm_obj(params, fun, W, data_matrix):
    moments = np.sum(fun(params, data_matrix), axis = 0)
    obj = np.dot(np.dot(moments.T, W), moments)
    return obj

In [30]:
# Find the MM estimator: first step
initial_params4 = np.zeros(4)
W_hat = np.eye(4)
model_4 = minimize(gmm_obj, initial_params4, args=(mm_fn, W_hat, mm_data), 
                   method='Nelder-Mead', options={'disp': True, 'maxiter': 10000})

Optimization terminated successfully.
         Current function value: 1139.447593
         Iterations: 148
         Function evaluations: 261


In [31]:
# Show the results
print(model_4.x)

[ 0.0950812  -0.04586472 -0.18883126  0.10889721]


In [32]:
# Update weighting matrix
err = np.matrix(mm_fn(model_4.x, mm_data))
S = np.dot(err.T, err) / mm_data.shape[0]
# Use the pseudo-inverse calculated by SVD because
# VCV2 is ill-conditioned
W_hat2 = lin.pinv(S)
W_hat2 = np.array(W_hat2)
W_hat2

array([[ 9.26729281e+01,  1.01326698e+02, -6.12120509e+00,
        -4.61426413e+00],
       [ 1.01326698e+02,  4.63544960e+02, -5.92491646e+00,
        -1.61019497e+01],
       [-6.12120509e+00, -5.92491646e+00,  1.05884325e+00,
        -1.86519470e-01],
       [-4.61426413e+00, -1.61019497e+01, -1.86519470e-01,
         9.30982396e-01]])

###### NOTE: lin.pinv stands for the pseudoinverse function provided by the numpy.linalg module. The pseudoinverse is used to compute the Moore-Penrose inverse of a matrix, which is a generalization of the matrix inverse for non-square matrices or for matrices that are singular or ill-conditioned. In other words, it calculates an approximate inverse for matrices that may not have a true inverse. This function is commonly used in numerical computations and linear algebra applications.

In [33]:
# Second step of optimization
model_41 = minimize(gmm_obj, model_4.x, args=(mm_fn, W_hat2, mm_data),
                    method='Nelder-Mead', options={'disp': True, 'maxiter': 10000})

Optimization terminated successfully.
         Current function value: 0.000007
         Iterations: 225
         Function evaluations: 388


In [34]:
# Show the results
print(model_41)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 7.148443809569187e-06
             x: [ 2.233e+00 -2.077e+00 -3.322e-01  1.326e-01]
           nit: 225
          nfev: 388
 final_simplex: (array([[ 2.233e+00, -2.077e+00, -3.322e-01,  1.326e-01],
                       [ 2.233e+00, -2.077e+00, -3.322e-01,  1.326e-01],
                       ...,
                       [ 2.233e+00, -2.077e+00, -3.322e-01,  1.326e-01],
                       [ 2.233e+00, -2.077e+00, -3.322e-01,  1.326e-01]]), array([ 7.148e-06,  7.898e-06,  3.105e-05,  3.246e-05,
                        3.455e-05]))


In [35]:
# Print coefficients
print(model_41.x)

[ 2.23329015 -2.07712488 -0.33221593  0.13257241]


In [36]:
### Gradient function
def gmm_moments1(params):
    # Select data for X [N x K]
    X = mm_data[:, 1:]
    # Select data for y [N x 1]
    y = mm_data[:, 0]
    
    # (i).Calculate the difference in representative utility for each decision maker. [N x 1]
    utility = np.dot(X, params)
    # (ii).Calculate logit choice probability for each decision maker. [N x 1]
    prob = 1 / (1 + np.exp(-utility))
    # (iii).Calculate econometric residuals [N x 1]
    residuals = y - prob
    
    # (iv).Calculate each of the 4 moments for each decision maker.
    moments = np.multiply(residuals.reshape(-1, 1), X)
    
    ### average moments
    moments = np.sum(moments, axis = 0)/mm_data.shape[0]
    return moments

In [37]:
### G0
x = np.array([model_41.x[0], model_41.x[1], model_41.x[2], model_41.x[3]],dtype=float)
G0_1 = scipy.optimize.approx_fprime(x, gmm_moments1, epsilon=1.4901161193847656e-08)
print(G0_1)

[[-2.07700401e-01 -8.07639382e-02 -2.15649444e+00 -2.85728525e+00]
 [-8.07639384e-02 -3.80658903e-02 -8.90438141e-01 -1.23583943e+00]
 [-2.15649440e+00 -8.90438116e-01 -2.36198930e+01 -3.07944980e+01]
 [-2.85728521e+00 -1.23583938e+00 -3.07944979e+01 -4.26268390e+01]]


In [42]:
# udpate S
err = np.matrix(mm_fn(model_41.x, mm_data))
S2 = np.dot(err.T, err) / mm_data.shape[0]
print(S2)

[[2.06434195e-01 8.00161938e-02 2.15149485e+00 2.82065304e+00]
 [8.00161938e-02 3.76937116e-02 8.93191046e-01 1.22170163e+00]
 [2.15149485e+00 8.93191046e-01 2.39020466e+01 3.07340942e+01]
 [2.82065304e+00 1.22170163e+00 3.07340942e+01 4.19723104e+01]]


In [43]:
#### VCOV
VCOV_1=np.dot(np.dot(G0_1.T, lin.pinv(S2)), G0_1)
VCOV_1=lin.pinv(VCOV_1)/mm_data.shape[0]
print(VCOV_1)

[[ 1.40792994e-01  1.11341338e-01 -9.90007230e-03 -5.55771050e-03]
 [ 1.11341338e-01  5.15500200e-01 -6.79293338e-03 -1.74810486e-02]
 [-9.90007230e-03 -6.79293338e-03  1.43858161e-03 -1.68655283e-04]
 [-5.55771050e-03 -1.74810486e-02 -1.68655283e-04  1.01940178e-03]]


In [44]:
### Point Estimates and Standard errors
print(model_41.x)
print(np.sqrt(np.diag(VCOV_1)))

[ 2.23329015 -2.07712488 -0.33221593  0.13257241]
[0.37522393 0.71798343 0.03792864 0.03192807]


Interpretation:
- Constant coefficient (2.23): it generates 2.23 "utils" of utility using car to go to campus when other independent variabels equal to zero
- Coefficient of cost.car (-2.08): an increase in cost of driving a car by one USD is associated with a decrease in the utiliy by 2.08
- Coefficient of time.car (-0.33): an increase in driving time by 1 minute is associated with a decrese in the utility by 0.33
- Coefficient of time.bus (0.13): an increase in bus travel time by 1 minute is associated with an increase in utility by 0.13

The choice to drive to campus during winter is modeled using the instruments described above in a generalized method of moments framework. The representative utility to student $n$ from driving and taking the bus, respectively, is represented as:

$$ 
V_{nc} = \alpha + \beta C_{nc} + \gamma_{car} T_{nc} \ 
$$

$$
V_{nb} = \gamma_{bus} T_{nb} \
$$

Here, $ C_{nj} $ represents the cost to student $n$ of alternative $j$, $T_{nj}$ represents the time for student $n$ of alternative $ j$, and the $ \alpha $, $\beta $, and $\gamma $ parameters are to be estimated. The parameters of this model are estimated using the generalized method of moments, with moment conditions constructed using the instruments described above. Thus, there are five instruments considered: the constant term, price gas, snowfall, construction, and bus detour.

In [46]:
list(df_bi.columns)

['id',
 'mode',
 'time.car',
 'cost.car',
 'time.bus',
 'cost.bus',
 'price_gas',
 'snowfall',
 'construction',
 'bus_detour',
 'age',
 'income',
 'marital_status']

In [47]:
## Calculate GMM estimator for binary commuting to school choice with instruments
## Create dataset for use in GMM moment function
gmm_data = df_bi.assign(constant = 1).loc[:,['mode', 'constant', 'cost.car',
                                               'time.car','time.bus', 'price_gas', 'snowfall','construction','bus_detour']].to_numpy()
print(gmm_data)

[[1.   1.   0.79 ... 0.   0.   1.  ]
 [1.   1.   0.59 ... 0.   0.   0.  ]
 [0.   1.   1.62 ... 1.   0.   0.  ]
 ...
 [0.   1.   0.39 ... 1.1  1.   0.  ]
 [0.   1.   0.32 ... 3.6  1.   0.  ]
 [1.   1.   0.28 ... 0.   0.   0.  ]]


In [48]:
## Function to calculate moments for commuting choice
def gmm_fn(params, data_matrix):
    # Select data for X [N x K]
    X = data_matrix[:, 1:5]
    # Select data for y [N x 1]
    y = data_matrix[:, 0]

    # Select data for Z [N x L]
    Z = data_matrix[:, [1, 5, 6, 7, 8]]

    # Calculate representative utility of air conditioning [N x 1]
    utility = np.dot(X, params)
    # Calculate logit choice probability of air conditioning [N x 1]
    prob = 1 / (1 + np.exp(-utility))
    # Calculate econometric residuals [N x 1]
    residuals = y - prob
    # Create moment matrix [N x L]
    moments = np.multiply(residuals.reshape(-1, 1), Z)

    return moments

In [49]:
print(gmm_data[0,[1,5,6,7,8]])

[1.   2.44 0.   0.   1.  ]


In [50]:
# Find the GMM estimator: first step
initial_params5 = np.zeros(4)
W_hat = np.eye(5)

model_5 = minimize(gmm_obj, initial_params5, args=(gmm_fn, W_hat, gmm_data),
                   method='Nelder-Mead', options={'disp': True, 'maxiter': 10000})

Optimization terminated successfully.
         Current function value: 27.831065
         Iterations: 131
         Function evaluations: 232


In [51]:
print(model_5.x)

[ 0.03754504  0.16314731 -0.30445596  0.20318423]


In [52]:
# Two-step weighting matrix
err2 = np.matrix(gmm_fn(model_5.x, gmm_data))
S2_1 = np.dot(err2.T, err2) / gmm_data.shape[0]
# Use the pseudo-inverse calculated by SVD because
# VCV2 is ill-conditioned
W_hat2_2 = lin.pinv(S2_1)
W_hat2_2 = np.array(W_hat2_2)
print(W_hat2_2)

[[ 2.67234758e+03 -1.06594022e+03 -1.70708383e-01  3.02985147e+01
  -2.02624614e+01]
 [-1.06594022e+03  4.26144242e+02 -2.52784924e-01 -1.38252210e+01
   6.45421506e+00]
 [-1.70708383e-01 -2.52784924e-01  7.39379366e-01 -1.05033590e-01
  -3.80540256e-01]
 [ 3.02985147e+01 -1.38252210e+01 -1.05033590e-01  5.01373604e+01
  -3.04456878e+00]
 [-2.02624614e+01  6.45421506e+00 -3.80540256e-01 -3.04456878e+00
   4.40020581e+01]]


In [53]:
# Find the GMM estimator
model_52 = minimize(gmm_obj, model_5.x, args=(gmm_fn, W_hat2_2, gmm_data),
                    method='Nelder-Mead', options={'disp': True, 'maxiter': 10000})

Optimization terminated successfully.
         Current function value: 5.516457
         Iterations: 231
         Function evaluations: 384


In [54]:
# Show the results
print(model_52)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 5.51645710701726
             x: [ 2.910e+00 -3.981e+00 -3.508e-01  1.502e-01]
           nit: 231
          nfev: 384
 final_simplex: (array([[ 2.910e+00, -3.981e+00, -3.508e-01,  1.502e-01],
                       [ 2.910e+00, -3.981e+00, -3.508e-01,  1.502e-01],
                       ...,
                       [ 2.910e+00, -3.981e+00, -3.508e-01,  1.502e-01],
                       [ 2.910e+00, -3.981e+00, -3.508e-01,  1.502e-01]]), array([ 5.516e+00,  5.516e+00,  5.516e+00,  5.516e+00,
                        5.516e+00]))


In [55]:
# Print coefficients
print(model_52.x)

[ 2.90992003 -3.98133682 -0.35079736  0.1502019 ]


In [56]:
### Gradient function
def gmm_moments2(params):
    # Select data for X [N x K]
    X = gmm_data[:, 1:5]
    # Select data for y [N x 1]
    y = gmm_data[:, 0]
    # Select data for Z [N x L]
    Z = gmm_data[:, [1, 5, 6, 7, 8]]
    
    # (i).Calculate the difference in representative utility for each decision maker. [N x 1]
    utility = np.dot(X, params)
    # (ii).Calculate logit choice probability for each decision maker. [N x 1]
    prob = 1 / (1 + np.exp(-utility))
    # (iii).Calculate econometric residuals [N x 1]
    residuals = y - prob
    # (iv).Calculate each of the 4 moments for each decision maker.
    moments = np.multiply(residuals.reshape(-1, 1), Z)
    
    ### average moments
    moments = np.sum(moments, axis = 0)/gmm_data.shape[0]
    
    return moments


In [57]:
print(gmm_moments2(model_52.x))

[ 8.01177074e-06 -6.79959552e-06 -2.33333467e-03  1.42662917e-04
 -7.50087575e-06]


In [58]:
### G0
x = np.array([model_52.x[0], model_52.x[1], model_52.x[2], model_52.x[3]],dtype=float)
G0 = scipy.optimize.approx_fprime(x, gmm_moments2, epsilon=1.4901161193847656e-08)
print(G0)

[[-1.93637556e-01 -7.21864167e-02 -1.98878031e+00 -2.60548483e+00]
 [-4.84481088e-01 -1.80805458e-01 -4.97366922e+00 -6.51601679e+00]
 [-2.36061353e-01 -8.22246356e-02 -2.77867923e+00 -3.13302401e+00]
 [-1.98028785e-02 -6.41556943e-03 -2.79050495e-01 -2.53054795e-01]
 [-2.22849860e-02 -8.59778172e-03 -2.38644441e-01 -4.05365175e-01]]


In [59]:
# Udpate S
err2 = np.matrix(gmm_fn(model_52.x, gmm_data))
S2_2 = np.dot(err2.T, err2) / gmm_data.shape[0]
print(S2_2)

[[0.20970489 0.5259832  0.25945411 0.02231313 0.02304985]
 [0.5259832  1.32140202 0.65118365 0.05638621 0.05741262]
 [0.25945411 0.65118365 1.57867475 0.03156781 0.0419054 ]
 [0.02231313 0.05638621 0.03156781 0.02231313 0.00321318]
 [0.02304985 0.05741262 0.0419054  0.00321318 0.02304985]]


In [60]:
#### VCOV
VCOV=np.dot(np.dot(G0.T, lin.pinv(S2_2)), G0)
VCOV=lin.pinv(VCOV)/gmm_data.shape[0]
print(VCOV)

[[ 1.45155450e+01 -2.99597978e+01 -4.45005426e-01  9.01967380e-02]
 [-2.99597978e+01  6.47825932e+01  8.98420558e-01 -2.51400067e-01]
 [-4.45005426e-01  8.98420558e-01  1.50711611e-02 -3.28514439e-03]
 [ 9.01967380e-02 -2.51400067e-01 -3.28514439e-03  2.75461786e-03]]


In [61]:
### Point Estimates and Standard errors
print(model_52.x)
print(np.sqrt(np.diag(VCOV)))

[ 2.90992003 -3.98133682 -0.35079736  0.1502019 ]
[3.80992716 8.04876346 0.12276466 0.05248445]


### Interpretation

- 2.909 represents the effect of the constant term (alpha) in the utility equation for driving (Vnc), indicating the overall attractiveness of driving relative to taking the bus. The high standard error of 3.809 shows considerable uncertainty in this estimate.


- -3.981 reflects the impact of driving costs (Cnc) in the utility equation for driving (Vnc). A 1 dollar increase in driving cost decreases utility by 3.981. The high standard error of 8.048 suggests significant uncertainty.


- -0.350 represents the effect of driving time (Tnc) on the utility of driving (Vnc). Each additional minute of driving decreases utility by 0.350. The low standard error of 0.122 indicates low uncertainty.


- 0.150 indicates the effect of bus travel time (Tnb) on the utility of taking the bus (Vnb). Each additional minute of bus travel increases utility by 0.150. The low standard error of 0.052 suggests low uncertainty.

In [63]:
# Compute the test statistic
moments = np.sum(gmm_fn(model_52.x, gmm_data), axis = 0)/gmm_data.shape[0]
j_test = np.dot(np.dot(moments.T, lin.pinv(S2_2)), moments)*gmm_data.shape[0]
print(j_test)

[[0.00588846]]


The model is tested for correct specification by conducting an overidentifying restrictions test. The results of this test, along with the critical value (95th percentile), are reported. 

This test helps ascertain whether the moment conditions used in the estimation are consistent with the data. If the test statistic exceeds the critical value, it indicates that the model may be misspecified, suggesting potential issues with the underlying assumptions or the model structure. Conversely, if the test statistic is below the critical value, it suggests that the model is correctly specified, providing more confidence in the estimated parameters.

In [64]:
# Degrees of freedom (L - K)
from scipy.stats import chi2
chi2.ppf(0.95, 1)

3.841458820694124

In [65]:
print(1 - chi2.cdf(np.array(j_test)[0][0], 1))

0.938833336107636


After performing the ovidentifying restrictions test, we find that the test statistics <  critical value at 95th percentile.
P-value > 0.05 which indicates that we fail to reject the null hypothesis. 
Hence, we reach the conclusion that out model is not overidentified, infact, it is correctly specified.